### Finding small molecule inhibitors for COVID-19 Disease Map (DM) pathways

Choose the name of the COVID-19 DM pathway to target

In [9]:
pathway_name = 'Interferon 1 pathway'

Fetch the relevant pathway model from MMINERVA, and extract model elements

In [1]:
from covid_19.disease_maps.minerva_client import *
from covid_19.disease_maps.id_mapping_minerva import *

# Some basic elements of the COVID-19 DM
config = get_config(default_map_name)
project_id = get_project_id_from_config(config)
models = get_models(project_id, default_map_name)

In [12]:
[m['name'] for m in models]

['overview',
 'Virus replication cycle',
 'PAMP signalling',
 'Interferon 1 pathway',
 'Orf3a protein interactions',
 'TGFbeta signalling',
 'Interferon lambda pathway',
 'Kynurenine synthesis pathway',
 'HMOX1 pathway',
 'Orf10 Cul2 pathway',
 'E protein interactions',
 'SARS-CoV-2 RTC and transcription',
 'JNK pathway',
 'Endoplasmatic Reticulum stress',
 'Apoptosis pathway',
 'Nsp14 and metabolism',
 'Coagulation pathway',
 'Nsp4 and Nsp6 protein interactions',
 'Pyrimidine deprivation',
 'Electron Transport Chain disruption',
 'Renin-angiotensin pathway',
 'Nsp9 protein interactions',
 'NLRP3 inflammasome activation']

In [2]:
model_ids = {m['name']: m['idObject'] for m in models if m['name'] != 'overview'}
model_elements = {model_name: get_model_elements(model_id, project_id, default_map_name)
                  for model_name, model_id in model_ids.items()}

Now construct the INDRA mappings (i.e., equivalents using INDRA-compatible grounding namespaces and IDs) for these elements

In [5]:
indra_mappings = {model_name: [indra_db_refs_from_minerva_refs(get_element_references(element))
                  for element in model_elements[model_name]] for model_name in model_ids}

INFO: [2021-03-04 23:59:42] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.8/bio_ontology.pkl


In [7]:
indra_groundings = {model_name: set(get_prioritized_db_refs_key(db_refs)
                                    for db_refs in indra_mappings[model_name] if db_refs)
                    for model_name in model_ids}

In [10]:
indra_groundings

{'Virus replication cycle': {('CHEBI', 'CHEBI:135632'),
  ('CHEBI', 'CHEBI:16113'),
  ('CHEBI', 'CHEBI:28815'),
  ('GO', 'GO:0005737'),
  ('GO', 'GO:0005768'),
  ('GO', 'GO:0005783'),
  ('GO', 'GO:0005791'),
  ('GO', 'GO:0005793'),
  ('GO', 'GO:0006412'),
  ('GO', 'GO:0019012'),
  ('GO', 'GO:0019013'),
  ('GO', 'GO:0031982'),
  ('GO', 'GO:0039718'),
  ('GO', 'GO:0070992'),
  ('GO', 'GO:0071360'),
  ('HGNC', '11876'),
  ('HGNC', '13557'),
  ('HGNC', '2527'),
  ('HGNC', '2537'),
  ('HGNC', '8004'),
  ('HGNC', '8568'),
  ('UP', 'P0DTC1'),
  ('UP', 'P0DTC2'),
  ('UP', 'P0DTC3'),
  ('UP', 'P0DTC4'),
  ('UP', 'P0DTC5'),
  ('UP', 'P0DTC6'),
  ('UP', 'P0DTC7'),
  ('UP', 'P0DTC8'),
  ('UP', 'P0DTC9'),
  ('UP', 'P0DTD1'),
  ('UP', 'P0DTD2'),
  ('UP', 'P0DTD3'),
  ('UP', 'P0DTD8'),
  (None, None)},
 'PAMP signalling': {('GO', 'GO:0005634'),
  ('GO', 'GO:0005737'),
  ('GO', 'GO:0005768'),
  ('HGNC', '10019'),
  ('HGNC', '11562'),
  ('HGNC', '11584'),
  ('HGNC', '11849'),
  ('HGNC', '12030'),
  ('H

Filter down to all the human genes

In [12]:
human_genes = {model_name: {entry for entry in indra_groundings[model_name] if entry[0] == 'HGNC'}
               for model_name in model_ids}

In [14]:
for model_name, genes in human_genes.items():
    print(model_name, len(genes))

Virus replication cycle 6
PAMP signalling 45
Interferon 1 pathway 47
Orf3a protein interactions 46
TGFbeta signalling 23
Interferon lambda pathway 50
Kynurenine synthesis pathway 39
HMOX1 pathway 53
Orf10 Cul2 pathway 9
E protein interactions 26
SARS-CoV-2 RTC and transcription 0
JNK pathway 15
Endoplasmatic Reticulum stress 71
Apoptosis pathway 22
Nsp14 and metabolism 70
Coagulation pathway 57
Nsp4 and Nsp6 protein interactions 40
Pyrimidine deprivation 26
Electron Transport Chain disruption 213
Renin-angiotensin pathway 21
Nsp9 protein interactions 90
NLRP3 inflammasome activation 15


We can now turn to the INDRA DB and find Inhibition statements targeting each of these human genes in the pathway of interest. We filter regulators to ones grounded to small molecule name spaces. We also filter out statements that have been curated as incorrect.

In [15]:
from indra.sources import indra_db_rest
from indra.tools import assemble_corpus as ac
try:
    from indra_db import get_db
    from indra_db.client.principal.curation import get_curations
    curs = get_curations(get_db('primary'))
except Exception:
    curs = []

In [17]:
inhibitors = {}
for model, entries in human_genes.items():
    for db_ns, db_id in entries:
        if (db_ns, db_id) in inhibitors:
            continue
        ip = indra_db_rest.get_statements(object=f'{db_id}@{db_ns}', stmt_type='Inhibition',
                                          ev_limit=200)
        stmts = [s for s in ip.statements if set(s.subj.db_refs) & {'CHEBI', 'PUBCHEM', 'CAS', 'CHEMBL', 'DRUGBANK'}]
        stmts = ac.filter_by_curation(stmts, curations=curs)
        inhibitors[(db_ns, db_id)] = stmts

INFO: [2021-03-05 00:12:06] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=11876@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:12:06] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:12:10] indra.tools.assemble_corpus - Filtering 34 statements with any incorrect curations...
INFO: [2021-03-05 00:12:10] indra.tools.assemble_corpus - 33 statements after filter...
INFO: [2021-03-05 00:12:10] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=8568@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:12:10] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:12:14] indra.tools.assemble_corpus - Filtering 51 statements with any incorrect curations...
INFO: [2021-03-05 00:12:14] indra.tools.assemble_corpus - 49 statements after filter...
INFO: [2021-03-

INFO: [2021-03-05 00:13:31] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:13:33] indra.tools.assemble_corpus - Filtering 1 statements with any incorrect curations...
INFO: [2021-03-05 00:13:33] indra.tools.assemble_corpus - 1 statements after filter...
INFO: [2021-03-05 00:13:33] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=17967@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:13:33] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:13:37] indra.tools.assemble_corpus - Filtering 48 statements with any incorrect curations...
INFO: [2021-03-05 00:13:37] indra.tools.assemble_corpus - 48 statements after filter...
INFO: [2021-03-05 00:13:37] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=6843@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05

INFO: [2021-03-05 00:13:44] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-03-05 00:13:44] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-03-05 00:13:44] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=18348@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:13:44] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:13:47] indra.tools.assemble_corpus - Filtering 33 statements with any incorrect curations...
INFO: [2021-03-05 00:13:47] indra.tools.assemble_corpus - 33 statements after filter...
INFO: [2021-03-05 00:13:47] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=7797@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:13:47] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05

INFO: [2021-03-05 00:15:06] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:15:08] indra.tools.assemble_corpus - Filtering 21 statements with any incorrect curations...
INFO: [2021-03-05 00:15:08] indra.tools.assemble_corpus - 21 statements after filter...
INFO: [2021-03-05 00:15:08] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=21158@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:15:08] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:15:10] indra.tools.assemble_corpus - Filtering 1 statements with any incorrect curations...
INFO: [2021-03-05 00:15:10] indra.tools.assemble_corpus - 1 statements after filter...
INFO: [2021-03-05 00:15:10] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=6881@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05

INFO: [2021-03-05 00:16:18] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:16:24] indra.tools.assemble_corpus - Filtering 23 statements with any incorrect curations...
INFO: [2021-03-05 00:16:24] indra.tools.assemble_corpus - 23 statements after filter...
INFO: [2021-03-05 00:16:24] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=18873@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 00:16:24] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 00:16:28] indra.tools.assemble_corpus - Filtering 31 statements with any incorrect curations...
INFO: [2021-03-05 00:16:28] indra.tools.assemble_corpus - 31 statements after filter...
INFO: [2021-03-05 00:16:28] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=9955@HGNC&offset=0&type=Inhibition
INFO: [2021-03-

...

INFO: [2021-03-05 01:01:26] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=2240@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:01:26] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:01:30] indra.tools.assemble_corpus - Filtering 6 statements with any incorrect curations...
INFO: [2021-03-05 01:01:30] indra.tools.assemble_corpus - 6 statements after filter...
INFO: [2021-03-05 01:01:30] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=8066@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:01:30] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:01:33] indra.tools.assemble_corpus - Filtering 2 statements with any incorrect curations...
INFO: [2021-03-05 01:01:33] indra.tools.assemble_corpus - 2 statements after filter...
INFO: [2021-03-05 01

INFO: [2021-03-05 01:02:19] indra.tools.assemble_corpus - 125 statements after filter...
INFO: [2021-03-05 01:02:19] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=19235@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:02:19] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:02:21] indra.tools.assemble_corpus - Filtering 7 statements with any incorrect curations...
INFO: [2021-03-05 01:02:21] indra.tools.assemble_corpus - 7 statements after filter...
INFO: [2021-03-05 01:02:21] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=28359@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:02:21] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:02:23] indra.tools.assemble_corpus - Filtering 1 statements with any incorrect curations...
INFO: [2021-03-0

INFO: [2021-03-05 01:03:45] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=6081@HGNC&offset=500&type=Inhibition
INFO: [2021-03-05 01:03:45] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:03:50] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=6081@HGNC&offset=1000&type=Inhibition
INFO: [2021-03-05 01:03:50] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:04:05] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=6081@HGNC&offset=1500&type=Inhibition
INFO: [2021-03-05 01:04:05] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:04:17] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=

INFO: [2021-03-05 01:05:25] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:05:28] indra.tools.assemble_corpus - Filtering 344 statements with any incorrect curations...
INFO: [2021-03-05 01:05:28] indra.tools.assemble_corpus - 344 statements after filter...
INFO: [2021-03-05 01:05:28] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=4653@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:05:28] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:05:30] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-03-05 01:05:30] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-03-05 01:05:30] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=9839@HGNC&offset=0&type=Inhibition
INFO: [2021-03-0

INFO: [2021-03-05 01:06:07] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=9192@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:06:07] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:06:09] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-03-05 01:06:09] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-03-05 01:06:09] indra.sources.indra_db_rest.util - query: http://localhost:5000/statements/from_agents?object=8743@HGNC&offset=0&type=Inhibition
INFO: [2021-03-05 01:06:09] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-03-05 01:06:11] indra.tools.assemble_corpus - Filtering 14 statements with any incorrect curations...
INFO: [2021-03-05 01:06:11] indra.tools.assemble_corpus - 14 statements after filter...
INFO: [2021-03-05 

We can now check how many targets there are small molecule inhibitors for

In [21]:
has_inhibitors = len([gene for gene, inhs in inhibitors.items() if inhs])
print('We found inhibitors for %d of %d targets' % (has_inhibitors, len(inhibitors)))

We found inhibitors for 635 of 802 targets


Now dump the results into a pickle, and a set of HTML files

In [22]:
import pickle
with open('inhibitors.pkl', 'wb') as fh:
    pickle.dump(inhibitors, fh)

In [49]:
readers = {'reach', 'sparser', 'trips', 'isi', 'rlimsp', 'eidos', 'medscan'}
def get_top_inhibitors(inhs):
    def get_sources(stmt):
        return {ev.source_api for ev in stmt.evidence}
    has_db_support = sorted([stmt for stmt in inhs if
                             get_sources(stmt) - readers],
                            key=lambda x: len(x.evidence),
                            reverse=True)
    no_db_support = sorted([stmt for stmt in inhs if
                           get_sources(stmt) <= readers],
                           key=lambda x: len(x.evidence),
                           reverse=True)
    return {'db_support': has_db_support[:5], 'no_db_support': no_db_support[:5]}
    

In [50]:
top_inhibitors = {target: get_top_inhibitors(inhs) for target, inhs in inhibitors.items()}

In [51]:
top_inhibitors

{('HGNC', '11876'): {'db_support': [],
  'no_db_support': [Inhibition(camostat(), TMPRSS2()),
   Inhibition(drug(), TMPRSS2()),
   Inhibition(nafamostat(), TMPRSS2()),
   Inhibition(bromhexine(), TMPRSS2()),
   Inhibition(chloroquine(), TMPRSS2())]},
 ('HGNC',
  '8568'): {'db_support': [Inhibition((R)-noradrenaline(), FURIN()),
   Inhibition(Pirfenidone(), FURIN()),
   Inhibition(dopaminoquinone(), FURIN()),
   Inhibition(Capric acid(), FURIN()),
   Inhibition(2-amino-2-deoxy-D-glucopyranose(), FURIN())], 'no_db_support': [Inhibition(iron atom(), FURIN()),
   Inhibition(citric acid(), FURIN()),
   Inhibition(luteolin(), FURIN()),
   Inhibition(flavonoid(), FURIN()),
   Inhibition(ketone(), FURIN())]},
 ('HGNC',
  '13557'): {'db_support': [Inhibition(enalapril(), ACE2()),
   Inhibition(D-CAPTOPRIL(), ACE2()),
   Inhibition(enalaprilat (anhydrous)(), ACE2()),
   Inhibition(ORE-1001(), ACE2()),
   Inhibition(N-(2-aminoethyl)-1-aziridine-ethanamine(), ACE2())], 'no_db_support': [Inhibition

In [53]:
from collections import Counter
from indra.databases import identifiers
from indra.statements.agent import default_ns_order
ns_order = default_ns_order + ['DRUGBANK', 'PUBCHEM', 'CAS' 'CHEMBL']
def get_table_rows(target, stmts_by_support):
    target_symbol = hgnc_client.get_hgnc_name(target[1])
    target_id = '%s:%s' % ('hgnc', target[1])
    rows = []
    for support_type in ['db_support', 'no_db_support']:
        for stmt in stmts_by_support[support_type]:
            drug = stmt.subj
            drug_name = drug.name
            drug_ns, drug_id = drug.get_grounding()
            if drug_ns:
                identifiers_ns = identifiers.get_identifiers_ns(drug_ns)
                drug_ns = identifiers_ns if identifiers_ns else drug_ns
                if drug_ns == 'chebi':
                    if not drug_id.startswith('CHEBI:'):
                        drug_id = 'CHEBI:' + drug_id
                drug_grounding = '%s:%s' % (drug_ns, drug_id)
            else:
                drug_grounding = ''
            pmids = sorted({ev.pmid for ev in stmt.evidence if ev.pmid})[:10]
            pmids_str = ', '.join(pmids) if pmids else ''
            sources = Counter([ev.source_api for ev in stmt.evidence])
            sources_str = ', '.join('%s:%s' % (source_name, source_count)
                                    for source_name, source_count in sources.most_common())
            row = [target_symbol, target_id, drug_name, drug_grounding, support_type, sources_str, pmids_str]
            rows.append(row)
    return rows

all_rows = []
for target, inh_stmts in top_inhibitors.items():
    all_rows += get_table_rows(target, inh_stmts)
with open('inhibitors.csv', 'w') as fh:
    writer = csv.writer(fh)
    writer.writerows(all_rows)

In [43]:
from indra.assemblers.html import HtmlAssembler
from indra.ontology.bio import bio_ontology
all_stmts = []
# Dump target-specific HTMLs
for gene_grounding, inhs_by_support in top_inhibitors.items():
    inhs = inhs_by_support['db_support'] + inhs_by_support['no_db_support']
    ha = HtmlAssembler(inhs, db_rest_url='https://db.indra.bio')
    ha.make_model()
    gene_name = bio_ontology.get_name(*gene_grounding)
    ha.save_model(f'{gene_name}.html')
    all_stmts += inhs
# Now dump for all inhibitors in a single file
ha = HtmlAssembler(all_stmts, db_rest_url='https://db.indra.bio')
ha.make_model()
ha.save_model(f'all_inhibitors.html')

INFO: [2021-03-05 10:44:34] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'68810458', '9863776'}
INFO: [2021-03-05 10:44:35] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:27363', '27363'}
INFO: [2021-03-05 10:44:35] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:27363', '27363'}
INFO: [2021-03-05 10:44:36] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'53481561', '4971'}
INFO: [2021-03-05 10:44:39] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'25941', 'CHEBI:25941'}
INFO: [2021-03-05 10:44:40] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:7549', '7549'}
INFO: [2021-03-05 10:44:40] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'50385', 'CHEBI:50385'}
INFO: [2021-03-05 10:44:41] indra.assemblers

NameError: name 'pathway_name' is not defined

In [44]:
# Now dump for all inhibitors in a single file
ha = HtmlAssembler(all_stmts, db_rest_url='https://db.indra.bio')
ha.make_model()
ha.save_model(f'all_inhibitors.html')

INFO: [2021-03-05 10:46:27] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:137113', 'CHEBI:95080'}
INFO: [2021-03-05 10:46:27] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'35475', 'CHEBI:35475'}
INFO: [2021-03-05 10:46:28] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:7549', '7549'}
INFO: [2021-03-05 10:46:28] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'53481561', '4971'}
INFO: [2021-03-05 10:46:28] indra.assemblers.html.assembler - Removing CAS from refs due to too many matches: {'130929-57-6', '116314-67-1'}
INFO: [2021-03-05 10:46:28] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'45353', 'CHEBI:45353'}
INFO: [2021-03-05 10:46:28] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'25941', 'CHEBI:25941'}
INFO: [2021-03-05 10:46:29] indra.

In [25]:
from indra.databases import hgnc_client

In [28]:
target_names = sorted(hgnc_client.get_hgnc_name(hid) for _, hid in inhibitors.keys())

In [29]:
target_names

['AARS2',
 'AATF',
 'ABCC1',
 'ABCG2',
 'ACAD9',
 'ACE',
 'ACE2',
 'ACMSD',
 'ACSL3',
 'ACTR2',
 'ACVR1',
 'ACVR1B',
 'ADA',
 'ADAM17',
 'ADK',
 'ADSL',
 'AFMID',
 'AGT',
 'AGTR1',
 'AGTR2',
 'AHR',
 'AIFM1',
 'AK1',
 'AK5',
 'AK7',
 'AK8',
 'AKR1B1',
 'AKT1',
 'ALAD',
 'ALAS1',
 'ALAS2',
 'ALB',
 'ALG11',
 'ALG5',
 'AMPD1',
 'AMPD3',
 'ANPEP',
 'AP2A2',
 'AP2M1',
 'APAF1',
 'APRT',
 'ARL6IP6',
 'ASIC1',
 'ATF2',
 'ATF4',
 'ATF6',
 'ATG5',
 'ATIC',
 'ATP13A3',
 'ATP1A1',
 'ATP1B1',
 'ATP5F1A',
 'ATP5F1B',
 'ATP5F1C',
 'ATP5F1D',
 'ATP5F1E',
 'ATP5F1EP2',
 'ATP5IF1',
 'ATP5MC1',
 'ATP5MC2',
 'ATP5MC3',
 'ATP5ME',
 'ATP5MF',
 'ATP5MG',
 'ATP5MJ',
 'ATP5MK',
 'ATP5PB',
 'ATP5PD',
 'ATP5PF',
 'ATP5PO',
 'ATP6AP1',
 'ATP6V0A1',
 'ATP6V0A2',
 'ATP6V0B',
 'ATP6V0C',
 'ATP6V0D1',
 'ATP6V0D2',
 'ATP6V0E1',
 'ATP6V0E2',
 'ATP6V1A',
 'ATP6V1B1',
 'ATP6V1B2',
 'ATP6V1C1',
 'ATP6V1C2',
 'ATP6V1D',
 'ATP6V1E1',
 'ATP6V1E2',
 'ATP6V1G1',
 'ATP6V1G2',
 'ATP6V1H',
 'B4GALT1',
 'BACH1',
 'BAD',
 'BAG6',

In [30]:
for target_name in target_names:
    print(target_name)

AARS2
AATF
ABCC1
ABCG2
ACAD9
ACE
ACE2
ACMSD
ACSL3
ACTR2
ACVR1
ACVR1B
ADA
ADAM17
ADK
ADSL
AFMID
AGT
AGTR1
AGTR2
AHR
AIFM1
AK1
AK5
AK7
AK8
AKR1B1
AKT1
ALAD
ALAS1
ALAS2
ALB
ALG11
ALG5
AMPD1
AMPD3
ANPEP
AP2A2
AP2M1
APAF1
APRT
ARL6IP6
ASIC1
ATF2
ATF4
ATF6
ATG5
ATIC
ATP13A3
ATP1A1
ATP1B1
ATP5F1A
ATP5F1B
ATP5F1C
ATP5F1D
ATP5F1E
ATP5F1EP2
ATP5IF1
ATP5MC1
ATP5MC2
ATP5MC3
ATP5ME
ATP5MF
ATP5MG
ATP5MJ
ATP5MK
ATP5PB
ATP5PD
ATP5PF
ATP5PO
ATP6AP1
ATP6V0A1
ATP6V0A2
ATP6V0B
ATP6V0C
ATP6V0D1
ATP6V0D2
ATP6V0E1
ATP6V0E2
ATP6V1A
ATP6V1B1
ATP6V1B2
ATP6V1C1
ATP6V1C2
ATP6V1D
ATP6V1E1
ATP6V1E2
ATP6V1G1
ATP6V1G2
ATP6V1H
B4GALT1
BACH1
BAD
BAG6
BAK1
BAMBI
BAX
BBC3
BCAP31
BCKDK
BCL2
BCL2L1
BCL2L11
BCS1L
BID
BLVRA
BLVRB
BMPR2
BRD2
BRD4
BST1
BTRC
C12orf65
C2
C3
C4A
C4B
C5
C6
C7
C8A
C8B
C8G
C9
CA12
CACTIN
CAD
CAND1
CANT1
CAPN1
CASP1
CASP10
CASP3
CASP4
CASP7
CASP8
CASP9
CAT
CCDC86
CCNT1
CD38
CDA
CDK5
CDK9
CEACAMP4
CFB
CFD
CGAS
CHCHD7
CHUK
CLCC1
CMA1
CMC1
CMPK1
COA1
COA3
COA4
COA5
COA6
COA7
COA8
COMT
COPS2
COPS4
COPS5
